<center>
<h2>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h2>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2020</h3>
<h4>Busqueda y Recomendación de Textos Legales - Preprocesamiento</h4>
</center>
</left>
<h4>Mentor: Claudio Sarate</h4>
</left>

### Introducción

En esta notebook, se implementara los pasos necesarios para el preprocesamiento de una de los archivos de muestra tomados de la base de Información Legislativa (INFOLEG) con la idea de generar un Corpus de texto para el posterior proceso. Para ello, comenzaremos con las importaciones pertinentes.

In [1]:
import collections
import multiprocessing
import os
import random
import re
import sys
import warnings
from pprint import pprint
from time import time

import numpy as np
import pandas as pd
import pprint as pprint

from IPython.display import display
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import defaultdict

pd.options.display.float_format = "{:.2f}".format

# %load_ext lab_black
# warnings.filterwarnings('ignore')

In [2]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield gensim.models.doc2vec.LabeledSentence(doc, [self.labels_list[idx]])

In [3]:
# Se verfica entorno de ejecución
in_colab = "google.colab" in sys.modules

if in_colab:
    from google.colab import drive

    drive.mount("/content/drive")
    BASE_DIR = "/content/drive/My Drive/Colab Notebooks/Data/"
else:
    BASE_DIR = "./Data"

## Inicio

#### Definición de directorios y nombres para el corpus a procesar

In [4]:
train_data_dir = "{}".format(os.sep).join([BASE_DIR, "Infoleg/"])
train_data = train_data_dir + "Infoleg.cor"

In [5]:
display(train_data_dir)
display(train_data)

'./Data/Infoleg/'

'./Data/Infoleg/Infoleg.cor'

#### Procedimiento para barrido del directorio donde estan los archivos con los cuales generar el Corpus

In [6]:
try:
    train = next(os.walk(train_data_dir))
except StopIteration:
    pass  # Some error handling here

### Generacion del Corpus 

#### Concatenación de archivos

In [7]:
# Limpieza del texto con la opción remover stopwords


def review_to_wordlist(review, remove_stopwords=False):

    # Conversión de palabras a minúsculas y separación
    words = review.lower().split()
    
    review_text = " ".join(words)

    # Opcionalmente se remueven stop words (true por default)

    # Return a list of words
    return review_text

In [8]:
docLabels = []
row_list = []
i = 0

for folder, subfolders, filenames in os.walk(train_data_dir):
    for file in filenames:
        if file.endswith(".txt"):
            d = {folder}
            with open(folder + file) as f:
                if f.read():
                    f.seek(0)
                    d = f.read()
                    docLabels.append(folder + file)
                    row_list.append(review_to_wordlist(d))
                    i = i + 1
    break

In [9]:
print(u"Archivos txt de entrenamiento leidos %s \n" % i)

Archivos txt de entrenamiento leidos 2500 



In [10]:
df_train = pd.DataFrame(row_list)
df_train.head(10)

,0
0,"ministerio de trabajo, empleo y seguridad soci..."
1,administracion federal de ingresos publicos di...
2,administración federal de ingresos públicos ob...
3,(nota infoleg: norma abrogada por art. 2° de l...
4,"secretaria de ciencia, tecnologia e innovacion..."
5,ministerio de defensa decreto 524/2007 desígna...
6,banco central de la republica argentina ( b.c....
7,"ministerio de trabajo, empleo y seguridad soci..."
8,banco central de la republica argentina (b.c.r...
9,consejo federal de la juventud ley 26.227 créa...


### Generación del corpus

In [11]:
df_train.to_csv(train_data, sep=" ")